In [1]:
import os

import numpy as np
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from utils.data_load import KittiDataset
from model.ensemblenet_model import EnsembleNet
import time
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [3]:
Num_Class = 2
Num_Channel = 3
Model_Name = 'segnet'
Scale = 0.5
Threshold = 0.5

imgdir = 'data/data_road/testing/image_2/'
in_files = os.listdir(imgdir)

In [4]:
def predict_img(net,
                full_img,
                device,
                scale_factor=1,
                out_threshold=0.5):
    net.eval()
    #img = torch.from_numpy(BasicDataset.preprocess(None, full_img, scale_factor, is_mask=False))
    img = torch.from_numpy(KittiDataset.preprocess(None, full_img, [384, 1242], scale_factor, is_mask=False))
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)
    
    with torch.no_grad():
        #output = net(img).cpu()
        output = net(img)
        output = F.interpolate(output, (full_img.size[1], full_img.size[0]), mode='bilinear')
        if net.n_classes > 1:
            mask = output.argmax(dim=1)
        else:
            mask = torch.sigmoid(output) > out_threshold

    return mask[0].cpu().long().squeeze().numpy()

### E-Net

In [4]:
#net = UNet(n_channels=3, n_classes=args.classes, bilinear=args.bilinear)
net = EnsembleNet(Model_Name, Num_Channel, Num_Class)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net.to(device=device)
state_dict = torch.load('../trained_enet/checkpoint_epoch51.pth', map_location=device)
mask_values = state_dict.pop('mask_values', [0, 1])
net.load_state_dict(state_dict)

print('model loaded!')

start = time.time()
for i, filename in enumerate(in_files):
    img = Image.open(imgdir + filename)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=Scale,
                       out_threshold=Threshold,
                       device=device)
print('{} Predict Image 290 total Inference Time : {}'.format(Model_Name, time.time() - start))

model loaded!
enet Predict Image 290 total Inference Time : 19.632378339767456


### U-Net

In [9]:
net = EnsembleNet(Model_Name, Num_Channel, Num_Class)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net.to(device=device)
state_dict = torch.load('../trained_unet/checkpoint_epoch51.pth', map_location=device)
mask_values = state_dict.pop('mask_values', [0, 1])
net.load_state_dict(state_dict)

print('model loaded!')

start = time.time()
for i, filename in enumerate(in_files):
    img = Image.open(imgdir + filename)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=Scale,
                       out_threshold=Threshold,
                       device=device)
print('{} Predict Image 290 total Inference Time : {}'.format(Model_Name, time.time() - start))

model loaded!
unet Predict Image 290 total Inference Time : 13.551630020141602


### SegNet

In [5]:
net = EnsembleNet(Model_Name, Num_Channel, Num_Class)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net.to(device=device)
state_dict = torch.load('../trained_segnet/checkpoint_epoch51.pth', map_location=device)
mask_values = state_dict.pop('mask_values', [0, 1])
net.load_state_dict(state_dict)

print('model loaded!')

start = time.time()
for i, filename in enumerate(in_files):
    img = Image.open(imgdir + filename)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=Scale,
                       out_threshold=Threshold,
                       device=device)
print('{} Predict Image 290 total Inference Time : {}'.format(Model_Name, time.time() - start))

model loaded!


/opt/conda/conda-bld/pytorch_1670525553989/work/aten/src/ATen/native/cuda/MaxUnpooling.cu:48: max_unpooling2d_forward_kernel: block: [319,0,0], thread: [645,0,0] Assertion `maxind >= 0 && maxind < outputImageSize` failed.
/opt/conda/conda-bld/pytorch_1670525553989/work/aten/src/ATen/native/cuda/MaxUnpooling.cu:48: max_unpooling2d_forward_kernel: block: [319,0,0], thread: [651,0,0] Assertion `maxind >= 0 && maxind < outputImageSize` failed.
/opt/conda/conda-bld/pytorch_1670525553989/work/aten/src/ATen/native/cuda/MaxUnpooling.cu:48: max_unpooling2d_forward_kernel: block: [319,0,0], thread: [661,0,0] Assertion `maxind >= 0 && maxind < outputImageSize` failed.
/opt/conda/conda-bld/pytorch_1670525553989/work/aten/src/ATen/native/cuda/MaxUnpooling.cu:48: max_unpooling2d_forward_kernel: block: [319,0,0], thread: [609,0,0] Assertion `maxind >= 0 && maxind < outputImageSize` failed.
/opt/conda/conda-bld/pytorch_1670525553989/work/aten/src/ATen/native/cuda/MaxUnpooling.cu:48: max_unpooling2d_fo

RuntimeError: cuDNN error: CUDNN_STATUS_MAPPING_ERROR

In [3]:
net = EnsembleNet(Model_Name, Num_Channel, Num_Class)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net.to(device=device)
state_dict = torch.load('../trained_ensemble_voting/checkpoint_epoch51.pth', map_location=device)
mask_values = state_dict.pop('mask_values', [0, 1])
net.load_state_dict(state_dict)

print('model loaded!')

start = time.time()
for i, filename in enumerate(in_files):
    img = Image.open(imgdir + filename)

    mask = predict_img(net=net,
                       full_img=img,
                       scale_factor=Scale,
                       out_threshold=Threshold,
                       device=device)
print('{} Predict Image 290 total Inference Time : {}'.format(Model_Name, time.time() - start))

RuntimeError: Error(s) in loading state_dict for EnsembleNet:
	Missing key(s) in state_dict: "attention_unet.conv.weight", "attention_unet.conv.bias", "attention_segnet.conv.weight", "attention_segnet.conv.bias", "attention_enet.conv.weight", "attention_enet.conv.bias", "feature_enhancement.0.weight", "feature_enhancement.0.bias", "feature_enhancement.1.weight", "feature_enhancement.1.bias", "feature_enhancement.1.running_mean", "feature_enhancement.1.running_var", "feature_enhancement.3.weight", "feature_enhancement.3.bias", "feature_enhancement.4.weight", "feature_enhancement.4.bias", "feature_enhancement.4.running_mean", "feature_enhancement.4.running_var". 
	Unexpected key(s) in state_dict: "se_block_unet.excitation.0.weight", "se_block_unet.excitation.0.bias", "se_block_unet.excitation.2.weight", "se_block_unet.excitation.2.bias", "se_block_segnet.excitation.0.weight", "se_block_segnet.excitation.0.bias", "se_block_segnet.excitation.2.weight", "se_block_segnet.excitation.2.bias", 